# Projected Distribution Shift Demo

本笔记本展示三种 1D 轴上的策略变化可视化方式：

1) 投影分布示意曲线（混合高斯）。
2) 离散候选集上的策略分布（棒棒糖图）。
3) 语义投影的一维 KDE（示例数据）。

数据路径：`final_results/data/policy_projection_demo`。

数据格式：
- `discrete_candidates.csv`
  - 必需列：`candidate_id`, `quality`, `logp_pre`, `logp_post`
  - 说明：同一候选集合上，训练前/后的 logprob。
- `semantic_projection.csv`
  - 必需列：`sample_id`, `z_pre`, `z_post`
  - 说明：样本在 1D 语义投影轴上的位置（训练前/后）。

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data_dir = Path('..') / 'data' / 'policy_projection_demo'

def mix_gauss(x, comps):
    """Return a normalized mixture of Gaussians over x."""
    y = np.zeros_like(x, dtype=float)
    for weight, mean, std in comps:
        y += weight * np.exp(-0.5 * ((x - mean) / std) ** 2) / (std * np.sqrt(2 * np.pi))
    return y

def gaussian_kde_1d(x, samples, bandwidth):
    """Simple Gaussian KDE without external deps."""
    diffs = (x[:, None] - samples[None, :]) / bandwidth
    kernel = np.exp(-0.5 * diffs**2) / (bandwidth * np.sqrt(2 * np.pi))
    return kernel.mean(axis=1)


## 1) 投影分布示意曲线

In [ ]:
x = np.linspace(-4, 4, 1200)

# 灰色：训练前/基座策略的投影分布
p0 = mix_gauss(x, [(0.55, -1.8, 0.9), (0.45, 1.2, 1.1)])

# 蓝色：SFT 风格（适度锐化）
psft = mix_gauss(x, [(0.45, -1.8, 1.0), (0.55, 1.5, 0.45)])

# 蓝色：Off-policy DPO/IPO 风格（更尖、更挤压）
pdpo = mix_gauss(x, [(0.85, -2.1, 0.22), (0.15, 2.4, 0.40)])

fig, axes = plt.subplots(1, 2, figsize=(10, 3.2))

# 面板 1：SFT
ax = axes[0]
ax.plot(x, p0, lw=3, alpha=0.45, color="gray", label="pre")
ax.plot(x, psft, lw=3, color="tab:blue", label="after")
ax.set_title("SFT")
ax.set_yticks([])
ax.set_xticks([])

# 标记 y_u^+（红箭头）
yu_plus = 1.5
yu_idx = np.argmin(np.abs(x - yu_plus))
ax.annotate(
    "",
    xy=(yu_plus, psft[yu_idx] * 1.15),
    xytext=(yu_plus, 0),
    arrowprops=dict(arrowstyle="-|>", lw=2.5, color="tab:red"),
)
ax.text(yu_plus + 0.05, 0.02, r"$y_u^+$")

# 面板 2：Off-policy DPO/IPO
ax = axes[1]
ax.plot(x, p0, lw=3, alpha=0.45, color="gray", label="pre")
ax.plot(x, pdpo, lw=3, color="tab:blue", label="after")
ax.set_title("Off-policy DPO / IPO")
ax.set_yticks([])
ax.set_xticks([])

# 标记 y*（虚线）
y_star = -2.1
ax.axvline(y_star, ls="--", lw=2, color="black")
ax.text(y_star - 0.05, 0.02, r"$y^*$")

# squeezing 区域注释
ax.annotate(
    "squeezing",
    xy=(0.2, 0.25),
    xytext=(0.2, 0.55),
    arrowprops=dict(arrowstyle="<->", lw=2),
    ha="center",
)

fig.suptitle("Projected Distribution Shift", fontsize=12, y=1.03)
fig.tight_layout()
plt.show()


## 2) 离散候选集策略（同一组 response）

In [ ]:
df = pd.read_csv(data_dir / 'discrete_candidates.csv')
quality = df['quality'].to_numpy()
logp0 = df['logp_pre'].to_numpy()
logpt = df['logp_post'].to_numpy()

def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    return exp_x / exp_x.sum()

p0 = softmax(logp0)
pt = softmax(logpt)

order = np.argsort(quality)
p0 = p0[order]
pt = pt[order]

fig, ax = plt.subplots(figsize=(9.5, 3.2))
idx = np.arange(len(p0))
ax.vlines(idx - 0.12, 0, p0, color="gray", alpha=0.55, linewidth=2.2)
ax.vlines(idx + 0.12, 0, pt, color="tab:blue", linewidth=2.2)
ax.set_title("离散候选集策略（同一组 response）")
ax.set_xlabel("候选 response 排名（按质量排序）")
ax.set_ylabel("概率质量")
ax.set_yticks([])
ax.set_xticks([])
ax.legend(["pre", "after"], frameon=False)
fig.tight_layout()
plt.show()


## 3) 语义投影 KDE（一维示例）

In [ ]:
df = pd.read_csv(data_dir / 'semantic_projection.csv')
pre = df['z_pre'].to_numpy()
post = df['z_post'].to_numpy()

x = np.linspace(-4, 4, 800)
kde_pre = gaussian_kde_1d(x, pre, bandwidth=0.35)
kde_post = gaussian_kde_1d(x, post, bandwidth=0.3)

fig, ax = plt.subplots(figsize=(9.5, 3.2))
ax.plot(x, kde_pre, color="gray", lw=3, alpha=0.5, label="pre")
ax.plot(x, kde_post, color="tab:blue", lw=3, label="after")
ax.set_title("语义投影 KDE（一维）")
ax.set_yticks([])
ax.set_xticks([])
ax.legend(frameon=False)
fig.tight_layout()
plt.show()
